In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
# !conda install -c conda-forge beautifulsoup4 --yes

## Download and Explore Dataset

In [2]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

In [3]:
table = soup.find('table')
td = table.find_all('td')

Postalcode = []
Borough = []
Neighborhood = []

for i in range(0, len(td), 3):
    Postalcode.append(td[i].text.strip())
    Borough.append(td[i+1].text.strip())
    Neighborhood.append(td[i+2].text.strip())

In [4]:
df_toronto = pd.DataFrame(data=[Postalcode, Borough, Neighborhood]).transpose()
df_toronto.columns = ['Postal Code', 'Borough', 'Neighborhood']
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Ignore cells with a borough that is Not assigned


In [5]:
df_toronto['Borough'].replace('Not assigned', np.nan, inplace=True)
df_toronto.dropna(subset=['Borough'], inplace=True)

df_toronto.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### If there is more than one neighborhood existing in one postal code area these neighborhoods will be combined into one row with the neighborhoods separated with a comma

In [6]:
df_toronto = df_toronto.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_toronto.columns = ['Postal Code', 'Borough', 'Neighborhood']
df_toronto.head(12)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [7]:
def impute_neirghborhood(row):
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
    
    return row

df_toronto = df_toronto.apply(impute_neirghborhood, axis=1)
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df_toronto.shape

(103, 3)